In [3]:
import os

os.listdir()
os.chdir('../..')
os.listdir()

['LICENSE.md',
 'convokit',
 '.DS_Store',
 'requirements.txt',
 'Makefile',
 'website',
 'datasets',
 'tests',
 'README.md',
 'setup.py',
 '.gitignore',
 'CONTRIBUTING.md',
 'examples',
 'doc',
 '.git']

In [4]:
import convokit
from convokit import Corpus, Toxicity, Sequence

corpus = convokit.Corpus(filename="datasets/reddit_coarse_discourse-corpus/corpus")

In [5]:
os.listdir()

['LICENSE.md',
 'convokit',
 '.DS_Store',
 'requirements.txt',
 'Makefile',
 'website',
 'datasets',
 'tests',
 'README.md',
 'setup.py',
 '.gitignore',
 'CONTRIBUTING.md',
 'examples',
 'doc',
 '.git']

In [6]:
corpus.print_summary_stats()

Number of Users: 63573
Number of Utterances: 115827
Number of Conversations: 9483


In [5]:
tox = Toxicity()
tox.transform(corpus)

In [12]:
for convo in corpus.iter_conversations():
    convo_scores = 0
    count = 0

    for utt in convo.iter_utterances():        
        '''
            rerunning this takes over a day for our 110k+ comments since it uses an api with limited query rate, 
            we'll load them from toxicity_dictionary.json that was pre-fetched,
            for others using our transformer, please run self.get_toxicity over the utterances on their corpus.
        '''

        #utt_score = self._get_toxicity(utt.text)


        utt_score = toxicity_scores[utt.id]
        
        convo_scores+=utt_score
        count+=1

        utt.add_meta('toxicity', utt_score)

    convo.add_meta('averagetoxicity', convo_scores/count)

NameError: name 'toxicity_scores' is not defined

In [9]:
list(corpus.iter_utterances())[0].meta['toxicity']

0.21341012

In [12]:
import random

#counter = 0
for convo in corpus.iter_conversations():
    
    temp_chain = []

    for utt in convo.iter_utterances():
        
        if utt.meta['post_depth'] == 2:
            temp_chain.append(utt.id)
    
    if len(temp_chain) > 0:
        
        convo.add_meta('chain', random.choice(temp_chain))
        
        uttid = random.choice(temp_chain)
        chosen_chain= []
        chosen_chain.append(uttid)

        utt = convo.get_utterance(uttid)
        while(utt.meta['post_depth'] > 0):
            if utt.reply_to in corpus.utterances:
                utt = convo.get_utterance(utt.reply_to)
                chosen_chain.append(utt.id)
            else:
                break
                
        #counter+=1
        chosen_chain.reverse()
        convo.add_meta('chain', chosen_chain)
    else:
        convo.add_meta('chain', None)
        
            
print(list(corpus.iter_conversations())[2000])

Conversation({'_owner': <convokit.model.corpus.Corpus object at 0x1050b0fd0>, '_id': 't3_1lddvq', '_utterance_ids': ['t3_1lddvq', 't1_cbye5qd', 't1_cbzrszq', 't1_cbzrvp1', 't1_cbzs1ow', 't1_cbzs2ow', 't1_cc07f3c', 't1_cc0vfqg', 't1_cc0w4ks', 't1_cc0whiu', 't1_cc0xpok'], '_usernames': None, '_meta': {'subreddit': 'showerthoughts', 'url': 'https://www.reddit.com/r/Showerthoughts/comments/1lddvq/what_if_everything_i_do_is_predetermined_and_i/', 'title': 'What if everything I do is predetermined and I have no control over it.', 'chain': ['t3_1lddvq', 't1_cbye5qd', 't1_cbzrszq']}})


In [7]:
sequence = Sequence()
sequence.transform(corpus)

In [33]:
print(list(corpus.iter_utterances())[2000])

Utterance({'id': 't1_d2h6eld', 'user': User([('name', 'Vaperius')]), 'root': 't3_4gfgf2', 'reply_to': 't1_d2h3fzj', 'timestamp': None, 'text': "You know a country was really trying for world domination when its harder to the pick one of the countries it hasn't had a military conflict with.", 'meta': {'post_depth': 2, 'majority_type': 'elaboration', 'majority_link': 't1_d2h3fzj', 'annotation-types': ['elaboration', 'elaboration', 'humor'], 'annotation-links': ['t1_d2h3fzj', 't1_d2h3fzj', 't1_d2h3fzj'], 'ups': 3}})


In [60]:
#top sequences

convos = corpus.iter_conversations()
utterance_corpus = corpus.utterances

sequences = []

for convo in convos:
    if convo.meta['chain']:
        sequences.append(form_chain(convo.meta['chain']))

def form_chain(convochain):
    chain = []
    for utt in convochain:
        if utterance_corpus[utt].meta['majority_type']:
            chain.append(utterance_corpus[utt].meta['majority_type'])
        else:
            chain.append("none")
    sequence = '-'.join(chain)
    return sequence

from collections import Counter
sequence_types = Counter(sequences)
sequence_types.most_common(20)

#print(sequence_types['question-answer-elaboration'])

for sequence in sequence_types:    
    for convo in convos:
        if convo.meta['chain']:
            chain_bucket = []
            chain_bucket_count = []
            form_chain(convo.meta['chain'])
                sequence_types[sequence]['toxicity'] =  



SyntaxError: invalid syntax (<ipython-input-60-43802d0b6348>, line 30)